In [2]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import tensorflow as tf

In [4]:
tf.__version__

'2.1.0-rc1'

In [5]:
import keras

Using TensorFlow backend.


In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = "/content/gdrive/My Drive/AIMLDataContent/bank.csv"

In [0]:
import pandas as pd
df = pd.read_csv(path)

In [69]:
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In the above dataframe, not all columns hold relevance in prediction of customers staying/exiting, hence we can get rid of such columns

In [70]:
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [0]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [0]:
y= df.pop('Exited')

In [0]:
X=df

In [74]:
X.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
dtype: object

In [0]:
X['Geography'] = pd.Categorical(X['Geography'])
X['Geography'] = X.Geography.cat.codes

In [76]:
X.dtypes

CreditScore          int64
Geography             int8
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
dtype: object

In [0]:
X['Gender'] = pd.Categorical(X['Gender'])
X['Gender'] = X.Gender.cat.codes

In [0]:
import numpy as np

In [0]:
#X_new=X.to_numpy()

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=5)

In [81]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7000, 10)
(3000, 10)
(7000,)
(3000,)


In [0]:
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [0]:
X_train = np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.backend import backend

In [0]:
tf.keras.backend.clear_session()
#Initialize model
model = tf.keras.models.Sequential()

#normalize data
model.add(tf.keras.layers.BatchNormalization())

In [0]:
model.add(Dense(256, input_shape=(10,), activation='relu'))
model.add(tf.keras.layers.Dense(200, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))


#Add another hidden layer
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))

In [0]:
#Add OUTPUT layer
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [0]:
sgd = optimizers.SGD(lr=0.03, decay=1e-6, momentum=0.5,clipvalue=0.5,nesterov=True)
model.compile(optimizer=sgd,loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [122]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size = 32)

Train on 7000 samples, validate on 3000 samples
Epoch 1/50
7000/7000 [==============================] - 1s 160us/sample - loss: 0.5331 - accuracy: 0.7957 - val_loss: 0.4174 - val_accuracy: 0.8290
Epoch 2/50
7000/7000 [==============================] - 1s 94us/sample - loss: 0.4251 - accuracy: 0.8201 - val_loss: 0.3775 - val_accuracy: 0.8420
Epoch 3/50
7000/7000 [==============================] - 1s 91us/sample - loss: 0.3960 - accuracy: 0.8359 - val_loss: 0.3670 - val_accuracy: 0.8457
Epoch 4/50
7000/7000 [==============================] - 1s 93us/sample - loss: 0.3798 - accuracy: 0.8444 - val_loss: 0.3961 - val_accuracy: 0.8180
Epoch 5/50
7000/7000 [==============================] - 1s 94us/sample - loss: 0.3767 - accuracy: 0.8437 - val_loss: 0.3618 - val_accuracy: 0.8530
Epoch 6/50
7000/7000 [==============================] - 1s 98us/sample - loss: 0.3700 - accuracy: 0.8470 - val_loss: 0.3571 - val_accuracy: 0.8543
Epoch 7/50
7000/7000 [==============================] - 1s 94us/sampl

In [123]:
print(model.evaluate(X_train, y_train, verbose=0))
print(model.evaluate(X_test, y_test, verbose=0))

[0.31207561642783027, 0.86414284]
[0.3709749485254288, 0.855]
